<a href="https://colab.research.google.com/github/JonDennisClark/3DMBWSFPSwDBRLE/blob/main/COSC489_HW3_FineTuning_Qwen1_5B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install unsloth library
# Please restart session when asked to restart session.
# Rerun this code to make sure that all libraries are installed.

!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.2.7 requires tyro, which is not installed.
unsloth-zoo 2025.2.7 requires protobuf<4.0.0,

In [ ]:
# Load a pre-trained Qwen model using unsloth

from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-1.5B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer the following question.

### Question:
{}

### Response:
{}
"""
print(prompt_style)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer the following question.

### Question:
{}

### Response:
{}



In [ ]:
question = "Who was the 35th president in the US?"
print(prompt_style.format(question, ""))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer the following question.

### Question:
Who was the 35th president in the US?

### Response:




In [ ]:
# Inference: Let's ask the LLM to answer the question about the 35th president.
FastLanguageModel.for_inference(model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
# print(response)
print("-----------------------------------------------")
print(response[0].split("### Response:")[1])

-----------------------------------------------


The 35th president of the United States was Gerald Ford.<|endoftext|>


In [ ]:
# Install the pandas library

!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv("http://moa-lab.net/cosc489-s25/assignments/finetune_president_data.csv")

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer the following question.

### Question:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Cot"]
    outputs = examples["Answer"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True,)

print(dataset["text"][0])
print("-------------------------")
print(dataset["text"][33])
print("-------------------------")
print(dataset["text"][34])
print("-------------------------")
print(dataset["text"][46])

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer the following question.

### Question:
Who was the 1st president in the United States?

### Response:
There was no president ahead of her/him. George Washington was the 1st president of the United States, serving from 1789-1797, and belonged to the Federalist Party.<|endoftext|>
-------------------------
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer the following question.

### Question:
Who was the 34th president in the United States?

### Response:
The former 33rd president was Harry S. Truman, 34th president was after the stage. Dwight D. Eisenhower was the 34th president of the United States, serving from 1953-1961, and belonged to the Republican Party.<

In [ ]:
# Load a pre-trained Qwen model using unsloth again for fine-tuning

from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-1.5B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

ModuleNotFoundError: No module named 'unsloth'

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
from trl import SFTConfig, SFTTrainer

training_args = SFTConfig(output_dir="./output",
                          max_seq_length=max_seq_length,
                          max_steps = 60,
                          learning_rate = 2e-4)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=training_args,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/47 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/47 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/47 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/47 [00:00<?, ? examples/s]

In [ ]:
# Disable all WANDB setting.
import os
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"

import wandb
wandb.init(mode="disabled")

# Start to fine-tune the LLM
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 47 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,1.779400
2,1.805000
3,1.725600
4,1.590700
5,1.444600
6,1.336200
7,1.143500
8,1.030700
9,0.944900
10,0.798600


In [ ]:
# Let's ask the question again.
question = "Who was the 35th president of the United States?"

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response)
print(response[0].split("### Response:")[1])

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPlease answer the following question.\n\n### Question:\nWho was the 35th president of the United States?\n\n### Response:\n\nThe former 34th president was Dwight D. Eisenhower, 35th president was after the stage. John F. Kennedy was the 35th president of the United States, serving from 1961-1963, and belonged to the Democratic Party.<|endoftext|>']


The former 34th president was Dwight D. Eisenhower, 35th president was after the stage. John F. Kennedy was the 35th president of the United States, serving from 1961-1963, and belonged to the Democratic Party.<|endoftext|>
